In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()


,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,4.0,2.0,13.0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,3.0,5.0,0.0,14.0,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,3.0,3.0,0.0,5.0,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,3.0,4.0,2.0,10.0,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,3.0,4.0,0.0,13.0,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [8]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()
'''

'\n# List of categorical columns\nobject_cols = [col for col in features.columns if \'cat\' in col]\n\n# one-hot-encode categorical columns\nX = features.copy()\nX_test = test.drop([\'id\'], axis=1).copy()\n\noh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")\nX_ohe = oh_encoder.fit_transform(features[object_cols])\nX_test_ohe = oh_encoder.transform(test[object_cols])\n\nX_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])\nX_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])\n\nX = pd.concat([X, X_ohe], axis=1)\nX_test = pd.concat([X_test, X_test_ohe], axis=1)\nX = X.drop(object_cols, axis=1)\nX_test = X_test.drop(object_cols, axis=1)\n    \n# Preview the one-hot-encoded features\nX.head()\n'

In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [11]:
pseudo = pd.read_csv("../input/30dml-pseudo2/xgb_submission_k.txt")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [12]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [13]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
               'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS']),
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
          'max_depth':trial.suggest_int('max_depth', 2, 3),
           'learning_rate':trial.suggest_uniform('learning_rate', 2.2e-1, 3e-1),
          'n_estimators':N_ESTIMATORS,
        'max_bin':trial.suggest_int('max_bin', 890, 914),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 159, 167),
         'reg_lambda':trial.suggest_uniform('reg_lambda', 16.9, 18.9),
            'subsample': trial.suggest_uniform('subsample', 0.565, 0.585)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  ctb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_row(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_seed'] = inseed

    model = ctb.CatBoostRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        use_best_model=True,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE
       
    )


    ctb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, ctb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-26 01:58:19,812] A new study created in memory with name: no-name-7a561667-14b0-459d-bfcd-f6f13dbec0ec


0:	learn: 0.5925261	test: 0.7383143	best: 0.7383143 (0)	total: 87.7ms	remaining: 29m 13s
1000:	learn: 0.5557435	test: 0.7112667	best: 0.7112667 (1000)	total: 23.3s	remaining: 7m 22s
2000:	learn: 0.5521626	test: 0.7100354	best: 0.7100354 (2000)	total: 46s	remaining: 6m 53s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7098881342
bestIteration = 2338

Shrink model to first 2339 iterations.
0:	learn: 0.5925436	test: 0.7382819	best: 0.7382819 (0)	total: 24ms	remaining: 7m 59s
1000:	learn: 0.5558720	test: 0.7113829	best: 0.7113829 (1000)	total: 23.2s	remaining: 7m 20s
2000:	learn: 0.5522935	test: 0.7101581	best: 0.7101579 (1999)	total: 46.1s	remaining: 6m 54s
3000:	learn: 0.5502316	test: 0.7099695	best: 0.7099687 (2998)	total: 1m 9s	remaining: 6m 31s


[I 2021-08-26 02:00:34,997] Trial 0 finished with value: 0.7097346055236204 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 2, 'learning_rate': 0.279601242525599, 'max_bin': 905, 'min_data_in_leaf': 159, 'reg_lambda': 17.43179722655722, 'subsample': 0.5662006954882283}. Best is trial 0 with value: 0.7097346055236204.


Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.709966438
bestIteration = 3003

Shrink model to first 3004 iterations.
0:	learn: 0.5928542	test: 0.7385794	best: 0.7385794 (0)	total: 23.8ms	remaining: 7m 56s
1000:	learn: 0.5570244	test: 0.7120170	best: 0.7120170 (1000)	total: 23.5s	remaining: 7m 26s
2000:	learn: 0.5531884	test: 0.7104013	best: 0.7103981 (1999)	total: 46.6s	remaining: 6m 58s
3000:	learn: 0.5511909	test: 0.7100713	best: 0.7100554 (2977)	total: 1m 9s	remaining: 6m 34s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7099416832
bestIteration = 3611

Shrink model to first 3612 iterations.
0:	learn: 0.5928565	test: 0.7385492	best: 0.7385492 (0)	total: 25.5ms	remaining: 8m 28s
1000:	learn: 0.5571892	test: 0.7119723	best: 0.7119723 (1000)	total: 23.4s	remaining: 7m 24s
2000:	learn: 0.5533330	test: 0.7104494	best: 0.7104435 (1978)	total: 46.5s	remaining: 6m 58s
3000:	learn: 0.5513331	test: 0.7100238	best: 0.7100208 (2998)	total: 1m 9s	rem

[I 2021-08-26 02:03:29,365] Trial 1 finished with value: 0.7098013654912383 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 2, 'learning_rate': 0.2278495428579023, 'max_bin': 904, 'min_data_in_leaf': 163, 'reg_lambda': 17.201673617683802, 'subsample': 0.5846139827336116}. Best is trial 0 with value: 0.7097346055236204.


...

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7100469644
bestIteration = 2468

Shrink model to first 2469 iterations.
0:	learn: 0.5924615	test: 0.7382059	best: 0.7382059 (0)	total: 22.7ms	remaining: 7m 34s
1000:	learn: 0.5555082	test: 0.7111286	best: 0.7111247 (999)	total: 23.4s	remaining: 7m 23s
2000:	learn: 0.5519719	test: 0.7100792	best: 0.7100726 (1968)	total: 46.4s	remaining: 6m 57s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7099059578
bestIteration = 2279

Shrink model to first 2280 iterations.
0:	learn: 0.5924749	test: 0.7382115	best: 0.7382115 (0)	total: 23.2ms	remaining: 7m 43s
1000:	learn: 0.5556167	test: 0.7111401	best: 0.7111354 (997)	total: 23.1s	remaining: 7m 18s
2000:	learn: 0.5520674	test: 0.7100975	best: 0.7100956 (1906)	total: 46.6s	remaining: 6m 59s


[I 2021-08-26 03:01:58,926] Trial 28 finished with value: 0.7097096508212319 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 2, 'learning_rate': 0.2919789248977589, 'max_bin': 900, 'min_data_in_leaf': 162, 'reg_lambda': 18.494624466128073, 'subsample': 0.5666411344995719}. Best is trial 20 with value: 0.7095854568172137.


Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7098847492
bestIteration = 2726

Shrink model to first 2727 iterations.
0:	learn: 0.5925190	test: 0.7382567	best: 0.7382567 (0)	total: 23.7ms	remaining: 7m 54s
1000:	learn: 0.5557534	test: 0.7112698	best: 0.7112698 (1000)	total: 23.1s	remaining: 7m 18s
2000:	learn: 0.5521664	test: 0.7101312	best: 0.7101254 (1927)	total: 46.6s	remaining: 6m 58s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7098550246
bestIteration = 2634

Shrink model to first 2635 iterations.
0:	learn: 0.5925320	test: 0.7382622	best: 0.7382622 (0)	total: 23.1ms	remaining: 7m 42s
1000:	learn: 0.5558390	test: 0.7114255	best: 0.7114247 (999)	total: 23s	remaining: 7m 16s
2000:	learn: 0.5522729	test: 0.7102806	best: 0.7102608 (1905)	total: 46.3s	remaining: 6m 56s


[I 2021-08-26 03:04:17,664] Trial 29 finished with value: 0.7097777207029027 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 2, 'learning_rate': 0.28204529486470725, 'max_bin': 900, 'min_data_in_leaf': 162, 'reg_lambda': 18.48831949846108, 'subsample': 0.5665684859386865}. Best is trial 20 with value: 0.7095854568172137.


Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7100608546
bestIteration = 2793

Shrink model to first 2794 iterations.
Number of finished trials: 30
Best trial: {'bootstrap_type': 'MVS', 'max_depth': 2, 'learning_rate': 0.2658641073071089, 'max_bin': 897, 'min_data_in_leaf': 162, 'reg_lambda': 18.28415615971921, 'subsample': 0.5705244926914805}


In [15]:
study.best_params

{'bootstrap_type': 'MVS',
 'max_depth': 2,
 'learning_rate': 0.2658641073071089,
 'max_bin': 897,
 'min_data_in_leaf': 162,
 'reg_lambda': 18.28415615971921,
 'subsample': 0.5705244926914805}

# Log

====== Ordinal encoding =========

0.7104169071824754 no noise ver3

0.7100365351403024 no noise ver6 (narrow space)

0.7099791926764689 no noise ver7 (narrow space)

0.7099021667949135 no noise ver8 (narrow space)

0.7098886176837581 no noise ver10 (narrow space)

0.7098848674925469 no noise ver11 (narrow space) final

0.7146235117715432 random noise ver4

0.7139039381188639 row-wise noise ver5

 ====== One-hot encoding =========
 
0.7104321496790325 no noise ver2
 
 ========================================== 
 =========== Changed pseudo ===============

====== Ordinal encoding =========

0.7099758299240976 no noise ver14

0.7099757677030406 no noise ver15 (narrow space)

0.7097344112879561 no noise ver16 (narrow space)
 
0.7097006154877742 no noise ver17 (narrow space)
 
0.709608093816904 no noise ver18 (narrow space)
 